# Protein Fold Dataset Creator Dmeo

This Demo is a simple example of using Dataset operations to create a datset

## Imports

In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import col, when
from mmtfPyspark.ml import proteinSequenceEncoder
from mmtfPyspark.mappers import structureToPolymerChains
from mmtfPyspark.filters import containsLProteinChain
from mmtfPyspark.datasets import secondaryStructureExtractor
from mmtfPyspark.webfilters import pisces
from mmtfPyspark.io import MmtfReader

## Define addProteinFoldType Class

In [2]:
def addProteinFoldType(data, minThreshold, maxThreshold):
    '''
    Adds a column "foldType" with three major secondary structure class:
    "alpha", "beta", "alpha+beta", and "other" based upon the fraction of alpha/beta content.

    The simplified syntax used in this method relies on two imports:
        from pyspark.sql.functions import when
        from pyspark.sql.functions import col

    Attributes:
        data (Dataset<Row>): input dataset with alpha, beta composition
        minThreshold (float): below this threshold, the secondary structure is ignored
        maxThreshold (float): above this threshold, the secondary structure is ignored
    '''

    return data.withColumn("foldType", \
                           when((col("alpha") > maxThreshold) & (col("beta") < minThreshold), "alpha"). \
                           when((col("beta") > maxThreshold) & (col("alpha") < minThreshold), "beta"). \
                           when((col("alpha") > maxThreshold) & (col("beta") < minThreshold), "alpha+beta"). \
                           otherwise("other")\
                           )

## Configure Spark Context

In [3]:
conf = SparkConf() \
            .setMaster("local[*]") \
            .setAppName("ProteinFoldDatasetCreatorDemo")

sc = SparkContext(conf = conf)

## Read MMTF Hadoop sequence file

Create non-redundant set (<=40% seq. identity) if L-protein chains

In [4]:
path = "../../resources/mmtf_reduced_sample/"
sequenceIdentity = 40
resolution = 2.0

pdb = MmtfReader \
        .readSequenceFile(path, sc) \
        .filter(pisces(sequenceIdentity, resolution)) \
        .flatMap(structureToPolymerChains()) \
        .filter(pisces(sequenceIdentity, resolution)) \
        .filter(containsLProteinChain())

## Get secondary structure content

In [5]:
data = secondaryStructureExtractor.getDataset(pdb)

## Classify chains by secondary structure type

In [6]:
minThreshold = 0.05
maxThreshold = 0.15
data = addProteinFoldType(data, minThreshold, maxThreshold)

## Add Word2Vec encoded feature vector

In [7]:
encoder = proteinSequenceEncoder(data)
n = 2 # Create 2-grams
windowSize = 25 # 25-amino residue window size for Word2Vec
vectorSize = 50 # dimension of feature vector
data = encoder.overlappingNgramWord2VecEncode(n, windowSize, vectorSize).cache()

data.printSchema()
data.show(10)

root
 |-- structureChainId: string (nullable = false)
 |-- sequence: string (nullable = false)
 |-- alpha: float (nullable = false)
 |-- beta: float (nullable = false)
 |-- coil: float (nullable = false)
 |-- dsspQ8Code: string (nullable = false)
 |-- dsspQ3Code: string (nullable = false)
 |-- foldType: string (nullable = false)
 |-- ngram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)

+----------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------+--------------------+--------------------+
|structureChainId|            sequence|      alpha|      beta|      coil|          dsspQ8Code|          dsspQ3Code|foldType|               ngram|            features|
+----------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------+--------------------+--------------------+
|          1FEC.B|SRAYDLVVIGAGSGGLE...| 0.3

## Keep only a subset of relevant fields for futher processing

In [8]:
data = data.select("structureChainId", "alpha", "beta", "coil", "foldType", "features")

data.show(10)

+----------------+-----------+----------+----------+--------+--------------------+
|structureChainId|      alpha|      beta|      coil|foldType|            features|
+----------------+-----------+----------+----------+--------+--------------------+
|          1FEC.B| 0.32783505|0.27216494|       0.4|   other|[0.53596010226718...|
|          1FGY.A| 0.13385826|0.37007874|  0.496063|   other|[0.61664335574540...|
|          1FHG.A|0.029411765| 0.5980392|0.37254903|    beta|[0.52854526283986...|
|          1FIP.B| 0.72602737|       0.0| 0.2739726|   alpha|[0.54597675293376...|
|          1FIT.A| 0.34126985|0.25396827| 0.4047619|   other|[0.42864070159115...|
|          1FIU.D| 0.48951048|0.12937063| 0.3811189|   other|[0.44706012661519...|
|          1FL2.A|  0.2580645|0.30967742|0.43225807|   other|[0.72988615707045...|
|          1FLT.Y|0.031914894| 0.4787234| 0.4893617|    beta|[0.67545148377564...|
|          1FM0.D| 0.28395063|0.27160493|0.44444445|   other|[0.47113022189587...|
|   

## Terminate Spark Context

In [9]:
sc.stop()